In [ ]:
import graph_tool.all as gt 

g=gt.collection.data['lesmis'].copy()

In [ ]:
# read graph txt

f = open('g.txt','r')
lines=[[n for n in x.split()] for x in f.readlines()]
f.close()
lines
g = gt.Graph()
lines_=[]
for _ in lines:
    l=[]
    l.append(int(_[0]))
    l.append(int(_[1]))
    l.append(float(_[2]))
    lines_.append(l)
print(lines_)
g.add_vertex(1 + max([l[0] for l in lines_] + [l[1] for l in lines_]))
property_map=g.new_edge_property('float')
for l in lines_:
    g.add_edge(g.vertex(l[0]),g.vertex(l[1]))
    property_map[g.edge(g.vertex(l[0]),g.vertex(l[1]))]=l[2]

In [ ]:
for e in g.edges():
    print(e)
print(list(property_map))

In [ ]:
#g = gt.collection.data["lesmis"].copy()

import random as rd 

#q = g.new_ep("double", .98)   # edge uncertainties
N = g.num_vertices()
E = g.num_edges()
#e = g.edge(11, 36)
#q[e] = .5                     # ambiguous true edge
q=property_map
#e = g.add_edge(15, 73)
#q[e] = .5                     # ambiguous spurious edge

# We inititialize UncertainBlockState, assuming that each non-edge
# has an uncertainty of q_default, chosen to preserve the expected
# density of the original network:

q_default = (E - sum(q)) / ((N * (N - 1))/2 - E)

state = gt.UncertainBlockState(g, q=q, q_default=q_default)
print('done state')
# We will first equilibrate the Markov chain
gt.mcmc_equilibrate(state, wait=100, mcmc_args=dict(niter=10))
print('100')
# Now we collect the marginals for exactly 100,000 sweeps, at
# intervals of 10 sweeps:

u = None              # marginal posterior edge probabilities
bs = []               # partitions
cs = []               # average local clustering coefficient

def collect_marginals(s):
   global bs, u, cs
   print('before',s==state)
   u = s.collect_marginal(u)
   bstate = s.get_block_state()
   bs.append(bstate.levels[0].b.a.copy())
   cs.append(gt.local_clustering(s.get_graph()).fa.mean())
   print('u',u)
   print('bs',bs)
print('1000')
gt.mcmc_equilibrate(state, force_niter=1000, mcmc_args=dict(niter=10),
                    callback=collect_marginals)

eprob = u.ep.eprob


In [ ]:
import load_graph
import importlib
importlib.reload(load_graph)
import load_graph
import Expected_mod as ex 
import clustering_bayesian_ref
importlib.reload(clustering_bayesian_ref)
import clustering_bayesian_ref as br 
filename='g.txt'
g,property_map=load_graph.read_graph_tool_g(filename)
clustering=br.clustering(g,property_map)


# According to Figure 11: Expected modularity according to the different community strengths
## bayes

In [ ]:
# go through dic choose file automatically

import os
import importlib
import time

import Expected_mod as ex 
import clustering_bayesian_ref
importlib.reload(clustering_bayesian_ref)
import clustering_bayesian_ref as br 
import load_graph
from Expected_mod import Trans_C1
importlib.reload(load_graph)
import load_graph
path='mcp_acp_data//k10_l10//'
filelist=[i for i in os.listdir(path) if (i[-3:]=='txt' and i[10]=='1' and i[11]=='0' and i[12]=='_' )]
filelist.sort()
print(filelist)

# run weighted louvain, save clustering
k=10
l=10
value=[]
T=[]
clustering=[]
rep=5
#path='datasets//'
for graph in filelist:
    Emod=0
    current_cluster=[]
    for _ in range(rep):
        g,property_map=load_graph.read_graph_tool_g(path+graph)
        

        edge=[]
        p=list(property_map.a)
        
            
        for e in g.edges():
            
            edge.append((int(e.source()),int(e.target())))
            

        t1=time.time()
        cluster=br.clustering(g,property_map)
        t2=time.time()
        current_cluster.append(cluster)
        #clustering.append(cluster)
        Emod+=ex.APWP(edge,p,Trans_C1(cluster))
    clustering.append(current_cluster)
    print('----------graph: ',graph,'----------')
    print('cluster:',cluster)
    print('-----------Ex modularity',Emod,'-----------')
    value.append(Emod/rep)
    T.append(t2-t1)




# run ex mod, save time and value



# calculate AMI according to Figure 12: AMI score according to the community strengths

In [ ]:
# go through dic choose file automatically

import os
import importlib
import time

import Expected_mod as ex 
import clustering_bayesian_ref
importlib.reload(clustering_bayesian_ref)
import clustering_bayesian_ref as br 
import load_graph
from Expected_mod import Trans_C1
importlib.reload(load_graph)
import load_graph
from Expected_mod import Trans_C2
from sklearn.metrics.cluster import adjusted_mutual_info_score as ami
node=100
nomalized_cluster=[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

path='mcp_acp_data//k10_l10//'
filelist=[i for i in os.listdir(path) if (i[-3:]=='txt' and i[10]=='1' and i[11]=='0' and i[12]=='_' )]
filelist.sort()
print(filelist)

# run weighted louvain, save clustering
k=10
l=10
AMI=[]
T=[]
clustering=[]
rep=5
#path='datasets//'
for graph in filelist:
    ami_=0
    current_cluster=[]
    for _ in range(rep):
        
        g,property_map=load_graph.read_graph_tool_g(path+graph)
        

        edge=[]
        p=list(property_map.a)
        
            
        for e in g.edges():
            
            edge.append((int(e.source()),int(e.target())))
            

        t1=time.time()
        cluster=br.clustering(g,property_map)
        t2=time.time()
        current_cluster.append(cluster)
        #clustering.append(cluster)
        nor_cluster=nomalized_cluster
        c=Trans_C2(Trans_C1(cluster),node)
        ami_+=ami(nor_cluster,c)
    AMI.append(ami_/rep)


    clustering.append(current_cluster)
    print('----------graph: ',graph,'----------')
    print('cluster:',cluster)
    print('-----------AMI',ami_/rep,'-----------')
  




# run ex mod, save time and value



In [ ]:
print(AMI)
AMI_=[]
for _ in AMI:
    AMI_.append(float(_))
print(AMI_)

# calculate AMI according toFigure 13: AMI score according to the increasing number of clusters, community strength s = 0.7

In [ ]:
from Expected_mod import Trans_C1, APWP
import os
import importlib
import time
import Expected_mod as ex 
import load_graph
import importlib
import time
import clustering_bayesian_ref
importlib.reload(clustering_bayesian_ref)
import clustering_bayesian_ref as br 
from Expected_mod import Trans_C2
from sklearn.metrics.cluster import adjusted_mutual_info_score as ami


importlib.reload(load_graph)

path2='mcp_acp_data//l10_p0.3//datasets//'

filelist1=[i for i in os.listdir(path2) if i[-3:]=='txt'  ]
filelist1.sort()
print(filelist1)

NMI=[]
X_=[]
l=10
path='mcp_acp_data//l10_p0.3//datasets//'
for graph in filelist1:

    strr=[]
    flag=0
   
    for _ in graph:
        if _=='k':
            
            flag=1
            continue
        if flag==0:
            continue
        if _<'0' or _>'9':
           
            break
        strr.append(_)
    
    k=sum([int(strr[i])*(10**(len(strr)-(i+1))) for i in range(len(strr))])
    c=[i for i in range(k*l)]
    
    stad_cluster=[c[x:x+l] for x in range(0, len(c), l)]
   
    print(graph)
    g,property_map=load_graph.read_graph_tool_g(path+graph)

    current_nmi=0
    time=3
    for _ in range(time):
        cluster=br.clustering(g,property_map)
    
        current_nmi+=ami(Trans_C2(stad_cluster,k*l),cluster)
    NMI.append(current_nmi/time)
    X_.append(k)

# Experiment to low prob. according to Performances under Different Probability Distributions (RQ5)

In [ ]:
# go through dic choose file automatically

import os
import importlib
import time

import Expected_mod as ex 
import clustering_bayesian_ref
importlib.reload(clustering_bayesian_ref)
import clustering_bayesian_ref as br 
import load_graph
from Expected_mod import Trans_C1
importlib.reload(load_graph)
import load_graph
import os
from tkinter import Tcl
import networkx
    
path2='datasets//l10_p0.3_evolving_lowP1//'
filelists=[i for i in os.listdir(path2)]
filelists=Tcl().call('lsort','-dict',filelists)

# run weighted louvain, save clustering

Baye={}

for file in filelists:
    if file[-4:] =='.txt':
        file_list=file.split('_')
       
        k=[i for i in file_list if i[0]=='k'][0][1:]
        l=[i for i in file_list if i[0]=='l'][0][1:]

        g=load_graph.read_g(path2+file)
        g,property_map=load_graph.read_graph_tool_g(path2+file)
       

        edge=[]
        p=list(property_map.a)
 
        for e in g.edges():
            
            edge.append((int(e.source()),int(e.target())))
   
        t1=time.time()
        cluster=br.clustering(g,property_map)
        t2=time.time()
        print(type(cluster))

        Baye.update({k:cluster})



# run ex mod, save time and value



In [ ]:
print(Baye)
baye_clustering={}
l=10
for i in Baye:
   
    k=int(i)
    clus=Trans_C1(list(Baye[i]))
    baye_clustering.update({i:clus})


In [ ]:

import json
path='mcp_acp_data//l10_p0.3_evolving_lowP1//result//bayes//'
with open(path+'bayes_increaseK_lowP.json','w') as fp:
          json.dump(baye_clustering,fp,indent=4) # With indent=4 (pretty format)

In [ ]:
with open(path+'bayes_increaseK_lowP.json','r') as fp:
    data=json.load(fp)
print(data)

# Experiment of high prob. according to Performances under Different Probability Distribu- tions (RQ5)

## have completely same graph structures with low prob.

In [ ]:
# go through dic choose file automatically

import os
import importlib
import time

import Expected_mod as ex 
import clustering_bayesian_ref
importlib.reload(clustering_bayesian_ref)
import clustering_bayesian_ref as br 
import load_graph
from Expected_mod import Trans_C1
importlib.reload(load_graph)
import load_graph
import os
from tkinter import Tcl
import networkx
    
path2='datasets//l10_p0.3_evolving_highP1//'
filelists=[i for i in os.listdir(path2)]
filelists=Tcl().call('lsort','-dict',filelists)

# run weighted louvain, save clustering

Baye={}

for file in filelists:
    if file[-4:] =='.txt':
        file_list=file.split('_')
       
        k=[i for i in file_list if i[0]=='k'][0][1:]
        l=[i for i in file_list if i[0]=='l'][0][1:]

        g=load_graph.read_g(path2+file)
        g,property_map=load_graph.read_graph_tool_g(path2+file)
       

        edge=[]
        p=list(property_map.a)
 
        for e in g.edges():
            
            edge.append((int(e.source()),int(e.target())))
   
        t1=time.time()
        cluster=br.clustering(g,property_map)
        t2=time.time()
        print(type(cluster))

        Baye.update({k:cluster})



# run ex mod, save time and value



In [ ]:
print(Baye)
baye_clustering2={}
l=10
for i in Baye:
   
    k=int(i)
    clus=Trans_C1(list(Baye[i]))
    baye_clustering2.update({i:clus})


In [ ]:
import json
path='mcp_acp_data//l10_p0.3_evolving_highP1//result//bayes//'
with open(path+'bayes_increaseK_highP.json','w') as fp:
          json.dump(baye_clustering2,fp,indent=4) # Wit

In [ ]:
with open(path+'bayes_increaseK_highP.json','r') as fp:
    data=json.load(fp)
print(data)

# Experiment polarized graph according to Performances under Different Probability Distributions (RQ5)

In [ ]:
# go through dic choose file automatically

import os
import importlib
import time

import Expected_mod as ex 
import clustering_bayesian_ref
importlib.reload(clustering_bayesian_ref)
import clustering_bayesian_ref as br 
import load_graph
from Expected_mod import Trans_C1
importlib.reload(load_graph)
import load_graph
import os
from tkinter import Tcl
import networkx
    
path2='datasets//l50_k2_p0.18_polarized_graph//'
filelists=[i for i in os.listdir(path2)]
filelists=Tcl().call('lsort','-dict',filelists)

# run weighted louvain, save clustering

Baye={}

for file in filelists:
    if file[-4:] =='.txt':
        file_list=file.split('_')
       
        k=[i for i in file_list if i[0]=='k'][0][1:]
        l=[i for i in file_list if i[0]=='l'][0][1:]
        name=file_list[-2]
        print(k)
        print(l)
        print(name)

        g=load_graph.read_g(path2+file)
        g,property_map=load_graph.read_graph_tool_g(path2+file)
       

        edge=[]
        p=list(property_map.a)
 
        for e in g.edges():
            
            edge.append((int(e.source()),int(e.target())))
   
        t1=time.time()
        cluster=br.clustering(g,property_map)
        t2=time.time()
        print(file)


        Baye.update({name:cluster})



# run ex mod, save time and value



In [ ]:

baye_clustering3={}
l=10
for i in Baye:
   
   
    clus=Trans_C1(list(Baye[i]))
    baye_clustering3.update({i:clus})
baye_clustering3


## saving results

In [ ]:
import json
path='mcp_acp_data//l50_k2_p0.18_polarized_graph//result//bayes//'
with open(path+'bayes_polarized.json','w') as fp:
          json.dump(baye_clustering3,fp,indent=4) # Wit

# krogan2006-core &mips

In [ ]:

import os
import importlib
import time

import Expected_mod as ex 
import clustering_bayesian_ref
importlib.reload(clustering_bayesian_ref)
import clustering_bayesian_ref as br 
import load_graph
from Expected_mod import Trans_C1
importlib.reload(load_graph)
import load_graph
import os
from tkinter import Tcl
import networkx
    
path2='mcp_acp_data//krogan2006_core//intersec_mips//net//krogan2006_core_mips_net.txt'

# run weighted louvain, save clustering

Baye={}



g=load_graph.read_g(path2)
g,property_map=load_graph.read_graph_tool_g(path2)


edge=[]
p=list(property_map.a)

for e in g.edges():
    
    edge.append((int(e.source()),int(e.target())))

t1=time.time()
cluster=br.clustering(g,property_map)
t2=time.time()
k=len(cluster)


Baye.update({k:cluster})



# run ex mod, save time and value



In [ ]:
baye={}
l=10
for i in Baye:
   
   
    clus=Trans_C1(list(Baye[i]))
    baye.update({i:clus})
baye

In [ ]:
import json
path='mcp_acp_data//krogan2006_core//intersec_mips//baye_results//'
with open(path+'bayes2.json','w') as fp:
          json.dump(baye,fp,indent=4) # Wit

# krogan2006-extended &mips

In [ ]:

import os
import importlib
import time

import Expected_mod as ex 
import clustering_bayesian_ref
importlib.reload(clustering_bayesian_ref)
import clustering_bayesian_ref as br 
import load_graph
from Expected_mod import Trans_C1
importlib.reload(load_graph)
import load_graph
import os
from tkinter import Tcl
import networkx
    
path2='mcp_acp_data//krogan2006_extended//intersec_mips//krogan2006_extented_mips_net.txt'

# run weighted louvain, save clustering

Baye={}



g=load_graph.read_g(path2)
g,property_map=load_graph.read_graph_tool_g(path2)


edge=[]
p=list(property_map.a)

for e in g.edges():
    
    edge.append((int(e.source()),int(e.target())))

t1=time.time()
cluster=br.clustering(g,property_map)
t2=time.time()
k=len(cluster)


Baye.update({k:cluster})



# run ex mod, save time and value



In [ ]:
baye={}
l=10
for i in Baye:
   
   
    clus=Trans_C1(list(Baye[i]))
    baye.update({i:clus})
baye

In [ ]:
import json
path='mcp_acp_data//krogan2006_extended//intersec_mips//baye_results//'
with open(path+'bayes.json','w') as fp:
          json.dump(baye,fp,indent=4) # Wit

# Effect of Shifting Probabilities (RQ2).

In [ ]:
from Expected_mod import Trans_C1, APWP
import os
import importlib
import time
import Expected_mod as ex 
import load_graph
import importlib
import time
import clustering_bayesian_ref
importlib.reload(clustering_bayesian_ref)
import clustering_bayesian_ref as br 
from Expected_mod import Trans_C2
from sklearn.metrics.cluster import adjusted_mutual_info_score as ami


importlib.reload(load_graph)

path2='mcp_acp_data//l10_k10_p0.4_multiplied//new//datasets'

filelist1=[i for i in os.listdir(path2) if i[-3:]=='txt'  ]
filelist1.sort()
print(filelist1)

clustering_bayes={}
exp_mod=[]
X_=[]
l=10
k=10
multi=[0.0,0.1,0.2,0.3,0.4,0.5]
i=0
path='mcp_acp_data//l10_k10_p0.4_multiplied//new//datasets//'
for graph in filelist1:
    print(graph)
   
    
    g,property_map=load_graph.read_graph_tool_g(path+graph)
    edge=[]
    p=list(property_map.a)
    
        
    for e in g.edges():
        
        edge.append((int(e.source()),int(e.target())))

    current_exp_mod=0
    time=5
    temp_cluster=[]
    for _ in range(time):
        cluster=br.clustering(g,property_map)
        temp_cluster.append(cluster)
        current_exp_mod+=ex.APWP(edge,p,Trans_C1(cluster))
    clustering_bayes.update({multi[i]:temp_cluster})
    i+=1
    print(clustering_bayes)

In [ ]:
baye={}
l=10
for i in clustering_bayes:
    print(clustering_bayes[i])
    temp=[]
    for j in clustering_bayes[i]:
   
        clus=Trans_C1(list(j))
        temp.append(clus)
    baye.update({i:temp})
print(baye)

In [ ]:
import json
path='mcp_acp_data//l10_k10_p0.4_multiplied//new//baye_results//'
with open(path+'bayes.json','w') as fp:
          json.dump(baye,fp,indent=4) # Wit

In [ ]:
with open(path+'bayes.json','r') as fp:
    data=json.load(fp)
print(data)

### AMI - Effect of Shifting Probabilities (RQ2).

In [ ]:

from sklearn.metrics.cluster import adjusted_mutual_info_score as ami
path2='mcp_acp_data//l10_k10_p0.4_multiplied//new//datasets'

filelist1=[i for i in os.listdir(path2) if i[-3:]=='txt'  ]
filelist1.sort()
print(filelist1)

exp_mod=[]

l=10
k=10
multi=[1,10,2,4,6,8]
i=0
cc=[i for i in range(k*l)]
    
stad_cluster=[cc[x:x+l] for x in range(0, len(cc), l)]
path='mcp_acp_data//l10_k10_p0.4_multiplied//new//datasets//'
for graph in filelist1:
    print(graph)
    strr=[]
    flag=0
   
    for _ in graph:
        if _=='i':
            
            flag=1
            continue
        if flag==0:
            continue
        if _<'0' or _>'9':
           
            break
        strr.append(_)
    
    multi=multi=str(0)+'.'+str(strr[1])#sum([int(strr[i])*(10**(len(strr)-(i+1))) for i in range(len(strr))])

    g,property_map=load_graph.read_graph_tool_g(path+graph)
    edge=[]
    p=list(property_map.a)
    for e in g.edges():
        
        edge.append((int(e.source()),int(e.target())))

    temp_mod=0
   
    for c in data[str(multi)]:
        rep=len(data[str(multi)])
        clu=Trans_C1(c)
        print(c)
        temp_mod+=ami(Trans_C2(stad_cluster,k*l),Trans_C2(c,k*l))

    exp_mod.append(temp_mod/rep)
    print(multi,':',exp_mod)

    

### Expected value -Effect of Shifting Probabilities (RQ2)

In [ ]:

from sklearn.metrics.cluster import adjusted_mutual_info_score as ami
from Expected_mod import APWP
path2='mcp_acp_data//l10_k10_p0.4_multiplied//new//datasets'

filelist1=[i for i in os.listdir(path2) if i[-3:]=='txt'  ]
filelist1.sort()
print(filelist1)

exp_mod=[]

l=10
k=10

i=0
cc=[i for i in range(k*l)]
    
stad_cluster=[cc[x:x+l] for x in range(0, len(cc), l)]
path='mcp_acp_data//l10_k10_p0.4_multiplied//new//datasets//'
for graph in filelist1:
    print(graph)
    strr=[]
    flag=0
   
    for _ in graph:
        if _=='i':
            
            flag=1
            continue
        if flag==0:
            continue
        if _<'0' or _>'9':
           
            break
        strr.append(_)
    
    multi=multi=str(0)+'.'+str(strr[1])#sum([int(strr[i])*(10**(len(strr)-(i+1))) for i in range(len(strr))])

    g,property_map=load_graph.read_graph_tool_g(path+graph)
    edge=[]
    p=list(property_map.a)
    for e in g.edges():
        
        edge.append((int(e.source()),int(e.target())))

    temp_mod=0
   
    for c in data[str(multi)]:
        rep=len(data[str(multi)])
        clu=Trans_C1(c)
        print(c)
        temp_mod+=APWP(edge,p,clu)

    exp_mod.append(temp_mod/rep)
    print(multi,':',exp_mod)

    